In [1]:
import pandas as pd

In [2]:
#Read socioeconomic status per zipcode

In [3]:
socioeconomic_df = pd.read_csv("acs_socioeconomic_2019.csv")

In [4]:
#Read causal effect file(Alabama)

In [5]:
al_treatment_effect = pd.read_csv('AL_climate_causal_effect.csv')

In [6]:
al_treatment_effect.head()

,zip,1,2,3,4,5,6,elderlyReturns,returnsTotalwSalariesWages,returnsDependentCareCredit,returnsEducationCredit,risk_group,propensity_score,price_shift,y_0,y_0_corrected,y_1,y_1_corrected,treatment_effect,treatment_effect_no_corr
0,35007,0.282297,0.228070,0.153110,0.110048,0.188995,0.037480,0.228868,0.811802,0.300638,0.060606,True,0.602717,9.901744,6.697492,6.697492,5.629468,12.717830,6.020338,-1.068024
1,35014,0.296296,0.302469,0.154321,0.086420,0.123457,0.037037,0.283951,0.839506,0.265432,0.049383,False,0.606330,10.907354,3.659495,22.070478,8.108160,8.108160,-13.962318,4.448664
2,35016,0.293808,0.246377,0.160738,0.104084,0.159420,0.035573,0.284585,0.803689,0.271410,0.044796,True,0.607142,6.826111,4.460052,4.460052,6.744470,6.878937,2.418885,2.284419
3,35020,0.510983,0.309827,0.106358,0.041618,0.027746,0.003468,0.187283,0.833526,0.264740,0.050867,True,0.627598,13.432346,3.954134,3.954134,3.626657,19.250803,15.296669,-0.327477
4,35022,0.272643,0.245841,0.171904,0.107209,0.166359,0.036044,0.268022,0.817006,0.266174,0.064695,True,0.604240,6.155837,5.957761,5.957761,7.239478,5.446083,-0.511679,1.281716


In [7]:
#Merge by zip

In [8]:
socioeconomic_df['zip'] = socioeconomic_df['zip'].astype(str).str.zfill(5)
al_treatment_effect['zip'] = al_treatment_effect['zip'].astype(str).str.zfill(5)

In [9]:
al_socioeconomic_treatment_effect = pd.merge(socioeconomic_df, al_treatment_effect, on="zip")

In [10]:
#OLS regression treatment effect ~ socioeconomic variables

In [11]:
import statsmodels.api as sm

X = al_socioeconomic_treatment_effect[['pct_poverty', 'pct_black', 'pct_college_plus', 'pct_mortgage']]
X = sm.add_constant(X)
y = al_socioeconomic_treatment_effect['treatment_effect_no_corr']

model = sm.OLS(y, X).fit()
print(model.summary())

                               OLS Regression Results                               
Dep. Variable:     treatment_effect_no_corr   R-squared:                       0.363
Model:                                  OLS   Adj. R-squared:                  0.346
Method:                       Least Squares   F-statistic:                     21.90
Date:                      Thu, 08 May 2025   Prob (F-statistic):           2.51e-14
Time:                              20:45:27   Log-Likelihood:                -319.33
No. Observations:                       159   AIC:                             648.7
Df Residuals:                           154   BIC:                             664.0
Df Model:                                 4                                         
Covariance Type:                  nonrobust                                         
                       coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------

In [12]:
# Mean treatment effect

In [13]:
al_socioeconomic_treatment_effect['treatment_effect_no_corr'].mean()

1.4598943766728987

In [14]:
# Mean treatment effect: Lower 25% & Upper 25%

In [15]:
low_poverty = al_socioeconomic_treatment_effect[al_socioeconomic_treatment_effect['pct_poverty'] <= al_socioeconomic_treatment_effect['pct_poverty'].quantile(0.25)]

In [16]:
low_poverty['treatment_effect_no_corr'].mean()

1.6543041649454253

In [17]:
high_poverty = al_socioeconomic_treatment_effect[al_socioeconomic_treatment_effect['pct_poverty'] >= al_socioeconomic_treatment_effect['pct_poverty'].quantile(0.75)]

In [18]:
high_poverty['treatment_effect_no_corr'].mean()

1.0466658683839019

In [19]:
low_black = al_socioeconomic_treatment_effect[al_socioeconomic_treatment_effect['pct_black'] <= al_socioeconomic_treatment_effect['pct_black'].quantile(0.25)]

In [20]:
low_black['treatment_effect_no_corr'].mean()

1.9971116134690992

In [21]:
high_black = al_socioeconomic_treatment_effect[al_socioeconomic_treatment_effect['pct_black'] >= al_socioeconomic_treatment_effect['pct_black'].quantile(0.75)]

In [22]:
high_black['treatment_effect_no_corr'].mean()

1.9047813689788142

In [23]:
low_college = al_socioeconomic_treatment_effect[al_socioeconomic_treatment_effect['pct_college_plus'] <= al_socioeconomic_treatment_effect['pct_college_plus'].quantile(0.25)]

In [24]:
low_college['treatment_effect_no_corr'].mean()

2.042064019310156

In [25]:
high_college = al_socioeconomic_treatment_effect[al_socioeconomic_treatment_effect['pct_college_plus'] >= al_socioeconomic_treatment_effect['pct_college_plus'].quantile(0.75)]

In [26]:
high_college['treatment_effect_no_corr'].mean()

0.4775069230303398

In [27]:
low_mortgage = al_socioeconomic_treatment_effect[al_socioeconomic_treatment_effect['pct_mortgage'] <= al_socioeconomic_treatment_effect['pct_mortgage'].quantile(0.25)]

In [28]:
low_mortgage['treatment_effect_no_corr'].mean()

2.6376870524889564

In [29]:
high_mortgage = al_socioeconomic_treatment_effect[al_socioeconomic_treatment_effect['pct_mortgage'] >= al_socioeconomic_treatment_effect['pct_mortgage'].quantile(0.75)]

In [30]:
high_mortgage['treatment_effect_no_corr'].mean()

0.3064255530855851

In [31]:
#Get Alabama socioeconomic status

In [32]:
socioeconomic_df = pd.read_csv("acs_socioeconomic_2019.csv")

In [33]:
zipdata = pd.read_excel("uszips.xlsx")

In [34]:
socioeconomic_df['zip'] = socioeconomic_df['zip'].astype(str).str.zfill(5)
zipdata['zip'] = zipdata['zip'].astype(str).str.zfill(5)

In [35]:
alabama_zipdata = zipdata[zipdata['state_name'] == 'Alabama']

In [36]:
al_socioeconomic = pd.merge(socioeconomic_df, alabama_zipdata, on="zip")

In [37]:
al_socioeconomic = al_socioeconomic.dropna(subset=['pct_poverty', 'pct_black', 'pct_college_plus', 'pct_mortgage'])

In [38]:
#Cluster zip using socioeconomic variables

In [39]:
from sklearn.preprocessing import StandardScaler

In [40]:
features = ['pct_poverty', 'pct_black', 'pct_college_plus', 'pct_mortgage']

In [41]:
scaler = StandardScaler()
al_socioeconomic_scaled = scaler.fit_transform(al_socioeconomic[features])

In [42]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

In [43]:
#Clustering
range_n_clusters = [2, 3, 4, 5, 6, 7, 8, 9, 10]
for k in range_n_clusters: #try different numbers of clusters
    kmeans = KMeans(n_clusters=k, random_state=324)
    cluster_labels = kmeans.fit_predict(al_socioeconomic_scaled)
    #report average Silhouette score
    silhouette_avg = silhouette_score(al_socioeconomic_scaled, cluster_labels)
    print("For n_clusters ={},".format(k)+" the average silhouette_score is :{}".format(silhouette_avg))

C:\Users\user\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


For n_clusters =2, the average silhouette_score is :0.34206801037081536
For n_clusters =3, the average silhouette_score is :0.3272881728570336
For n_clusters =4, the average silhouette_score is :0.26290705134106146
For n_clusters =5, the average silhouette_score is :0.26987249251588613


C:\Users\user\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
C:\Users\user\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
C:\Users\user\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
C:\Users\user\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_

For n_clusters =6, the average silhouette_score is :0.2626123272582447
For n_clusters =7, the average silhouette_score is :0.2447964570451756
For n_clusters =8, the average silhouette_score is :0.25747706407528


C:\Users\user\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
C:\Users\user\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
C:\Users\user\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


For n_clusters =9, the average silhouette_score is :0.2369965315955513
For n_clusters =10, the average silhouette_score is :0.23279110874591585


C:\Users\user\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


In [44]:
kmeans = KMeans(n_clusters=2, random_state=324)
cluster_labels = kmeans.fit_predict(al_socioeconomic_scaled)

C:\Users\user\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


In [45]:
al_socioeconomic['cluster'] = cluster_labels

In [46]:
cluster_profile = al_socioeconomic.groupby('cluster')[features].mean()
print(cluster_profile)

         pct_poverty  pct_black  pct_college_plus  pct_mortgage
cluster                                                        
0           0.292483   0.622569          0.124702      0.371191
1           0.148811   0.131215          0.207962      0.498804


In [47]:
al_socioeconomic

,zip,pct_poverty,pct_black,pct_college_plus,pct_mortgage,lat,lng,city,state_id,state_name,...,density,county_fips,county_name,county_weights,county_names_all,county_fips_all,imprecise,military,timezone,cluster
0,36521,0.101103,0.059937,0.077958,0.581220,30.96241,-88.18526,Chunchula,AL,Alabama,...,17.7,1097,Mobile,"{""01097"": 100}",Mobile,1097,False,False,America/Chicago,1
1,36471,0.378319,0.661504,0.065350,0.205674,31.62616,-87.22861,Peterman,AL,Alabama,...,2.7,1099,Monroe,"{""01099"": 90.14, ""01035"": 9.86}",Monroe|Conecuh,01099|01035,False,False,America/Chicago,0
2,36476,0.000000,1.000000,0.000000,0.285714,31.35359,-86.54633,River Falls,AL,Alabama,...,41.3,1039,Covington,"{""01039"": 100}",Covington,1039,False,False,America/Chicago,0
3,36480,0.319263,0.118312,0.097500,0.266846,31.32976,-87.61125,Uriah,AL,Alabama,...,3.4,1099,Monroe,"{""01099"": 100}",Monroe,1099,False,False,America/Chicago,0
4,36513,0.825893,0.093750,0.088957,0.400000,31.15516,-87.99021,Calvert,AL,Alabama,...,7.6,1129,Washington,"{""01129"": 52.84, ""01097"": 47.16}",Washington|Mobile,01129|01097,False,False,America/Chicago,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
632,36453,0.188031,0.000835,0.113624,0.384389,31.19788,-86.14726,Kinston,AL,Alabama,...,9.8,1031,Coffee,"{""01031"": 52.25, ""01061"": 39.07, ""01039"": 8.68}",Coffee|Geneva|Covington,01031|01061|01039,False,False,America/Chicago,1
633,36467,0.191784,0.150765,0.131666,0.508535,31.27950,-86.25579,Opp,AL,Alabama,...,24.0,1039,Covington,"{""01039"": 94.16, ""01031"": 5.84}",Covington|Coffee,01039|01031,False,False,America/Chicago,1
634,36790,0.235955,0.752809,0.000000,0.118644,32.72758,-86.91070,Stanton,AL,Alabama,...,9.6,1021,Chilton,"{""01021"": 100}",Chilton,1021,False,False,America/Chicago,0
635,36832,0.393381,0.282729,0.446630,0.508937,32.59447,-85.58163,Auburn,AL,Alabama,...,170.1,1081,Lee,"{""01081"": 96.94, ""01087"": 3.06}",Lee|Macon,01081|01087,False,False,America/Chicago,1


In [48]:
al_socioeconomic.to_csv("al_socioeconomic.csv", index=False)